In [5]:
import os
import json
import glob
import random

In [6]:

class RawDataReader:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            self.data = json.load(file)

        self.id = self.data['ID']
        # self.file_identifier = self.data['patients_file_identifier']
        self.plan_details = self.extract_plan_details()

    def extract_plan_details(self):
        plan_details = []
        for plan in self.data['plans']:
            if plan['completed'] == True:
                plan_name = plan['name']
                planning_exam_name = plan['planning_examination']['name']
                planningCT_filename = plan['planning_examination']['dose_DICOM_filename']

# If you want only one opt per repeated CTs.
                evaluation_exams = []
                for exam in plan['evaluation_examinations']:
                    # Shuffle the list of optimization cases randomly
                    random.shuffle(exam['optimization_cases'])
                    
                    # Iterate over the shuffled optimization cases and select the first completed one
                    for opt_case in exam['optimization_cases']:
                        if opt_case.get('completed', True) and 'gradient_descent_result' in opt_case:
                            final_trans_coord = opt_case['gradient_descent_result'].get('final_translation_coordinate', {})
                            x = round(final_trans_coord.get('x', 0), 5)
                            y = round(final_trans_coord.get('y', 0), 5)
                            z = round(final_trans_coord.get('z', 0), 5)
                            exam_detail = {
                                'final_translation_coordinate': {'x': x, 'y': y, 'z': z},
                                'repeatedCT_filename': opt_case['gradient_descent_result'].get('dose_DICOM_filename', '')
                            }
                            evaluation_exams.append(exam_detail)
                            break  # Only select one optimization case per examination
                    # break  # Only select one examination per plan

# If you want to have all optimization cases uncomment this below.
                # evaluation_exams = []
                # for exam in plan['evaluation_examinations']:
                #     for opt_case in exam['optimization_cases']:
                #         if opt_case.get('completed', True) and 'gradient_descent_result' in opt_case:
                #             final_trans_coord = opt_case['gradient_descent_result'].get('final_translation_coordinate', {})
                #             x = round(final_trans_coord.get('x', 0), 5)
                #             y = round(final_trans_coord.get('y', 0), 5)
                #             z = round(final_trans_coord.get('z', 0), 5)
                #             exam_detail = {
                #                 # 'name': exam['name'],
                #                 'final_translation_coordinate': {'x': x, 'y': y, 'z': z},
                #                 'repeatedCT_filename': opt_case['gradient_descent_result'].get('dose_DICOM_filename', '')
                #             }
                #             evaluation_exams.append(exam_detail)
                        

                plan_details.append({
                    'plan_name': plan_name,
                    'planning_exam_name': planning_exam_name,
                    'planningCT_filename': planningCT_filename,
                    'evaluation_examinations': evaluation_exams
                })
            else:
                pass

        return plan_details

    def get_plan_details(self):
        return self.plan_details
    
# Usage as an example
file_path = '/data/shahpouriz/DBP_newDATA/results_DBP_OP038_Protons_20240301_151003.json'
treatment_data = RawDataReader (file_path)
# print(treatment_data.id)
plan_details = treatment_data.get_plan_details()
print(len(plan_details))
for plan_detail in plan_details:

    # print(plan_detail.get('plan_name'))
    # print(plan_detail.get('planningCT_filename'))
    for eval_exam in plan_detail['evaluation_examinations']:
        print(eval_exam['repeatedCT_filename'])

2
DBP_OP038/A1PHH/rCTp1/optimization_case_2/rtdose_rCTp1_1.2.752.243.1.1.20240228144442171.8610.22472.dcm
DBP_OP038/A1PHH/rCTp6/optimization_case_4/rtdose_rCTp6_1.2.752.243.1.1.20240228155620704.5087.69261.dcm
DBP_OP038/A1PHH/rCTp11/optimization_case_1/rtdose_rCTp11_1.2.752.243.1.1.20240228163610458.1621.61224.dcm
DBP_OP038/A1PHH/rCTp16/optimization_case_4/rtdose_rCTp16_1.2.752.243.1.1.20240228183349443.3429.68957.dcm
DBP_OP038/A1PHH/rCTp23/optimization_case_6/rtdose_rCTp23_1.2.752.243.1.1.20240228211145654.8317.19714.dcm
DBP_OP038/A1PHH/rCTp26/optimization_case_5/rtdose_rCTp26_1.2.752.243.1.1.20240228225855405.7543.66988.dcm
DBP_OP038/A1PHH/rCTp31/optimization_case_0/rtdose_rCTp31_1.2.752.243.1.1.20240228233953367.1148.89239.dcm
DBP_OP038/AAPHH/pCTp0/optimization_case_4/rtdose_pCTp0_1.2.752.243.1.1.20240229032856816.2336.31009.dcm
DBP_OP038/AAPHH/rCTp1/optimization_case_6/rtdose_rCTp1_1.2.752.243.1.1.20240229061842432.8738.31683.dcm
DBP_OP038/AAPHH/rCTp6/optimization_case_6/rtdose_rCT

--------------------

In [7]:

class DirectoryProcessor:
    def __init__(self, directory_path):
        self.directory_path = directory_path

    def process_files(self):
        
        # Search for json files matching the pattern
        file_pattern = os.path.join(self.directory_path, 'results_DBP_OP*.json')
        json_files = glob.glob(file_pattern)

        results = []
        for file_path in json_files:
            reader = RawDataReader(file_path)
            file_results = {
                'id': reader.id,
                'plan_details': reader.get_plan_details()
            }
            results.append(file_results)
        return results

# Usage
directory_path = '/home/shahpouriz/Data/DBP_newDATA'
processor = DirectoryProcessor(directory_path)
all_results = processor.process_files()

# for result in all_results:
#     print(result)

------------
# convert_dicom_to_nrrd

In [12]:
import os
import SimpleITK as sitk

class DICOMtoNRRDConverter:
    def __init__(self, directory_path, output_directory_proton, output_directory_foton):
        self.directory_path = directory_path
        self.output_directory_proton = output_directory_proton
        self.output_directory_foton = output_directory_foton
        self.directory_processor = DirectoryProcessor(directory_path)

    def convert_and_save(self):
        patient_data = self.directory_processor.process_files()
        for patient in patient_data:
            patient_id = patient['id']
            for plan_index, plan in enumerate(patient['plan_details']):
                if 'Not_Completed' in plan['plan_name']:
                    print('dgfd')
                    pass
                else:
                    # Determine the output path based on whether the plan is for proton or foton
                    output_path = self.output_directory_proton if 'P' in plan['plan_name'] else self.output_directory_foton
                    full_output_path = os.path.join(output_path, patient_id)
                    if not os.path.exists(full_output_path):
                        os.makedirs(full_output_path)
                    # Process files for this plan
                    plan_name = 'P' if 'P' in plan['plan_name'] else 'F'  # Assuming 'P' for proton and 'F' for foton
                    nrrd_path = self._process_dose_file(plan['planningCT_filename'], full_output_path, patient_id, plan_name, plan_index, "planningCT")
                    plan['planningCT_filename'] = nrrd_path or plan['planningCT_filename']
                    for exam_index, exam in enumerate(plan.get('evaluation_examinations', [])):
                        nrrd_path = self._process_dose_file(exam['repeatedCT_filename'], full_output_path, patient_id, plan_name, plan_index, "repeatedCT", exam_index)
                        exam['repeatedCT_filename'] = nrrd_path or exam['repeatedCT_filename']
        # Optional: Return the updated patient data for verification
        return patient_data

    def _process_dose_file(self, dose_dicom_filename, output_path, patient_id, plan_name, plan_index, exam_type, exam_index=None):
        
        # Constructing unique filename based on patient_id, plan, exam type, and an ordinal number
        if exam_index is not None:
            nrrd_filename = f"{patient_id}_{plan_name}{plan_index+1}_{exam_type}{exam_index+1}.nrrd"
            print(nrrd_filename)
            print(exam_type)
        else:
            nrrd_filename = f"{patient_id}_{plan_name}{plan_index+1}_{exam_type}.nrrd"
            print(nrrd_filename)
            print(exam_type)
        nrrd_path = os.path.join(output_path, nrrd_filename)
        full_dose_path = os.path.join(self.directory_path, dose_dicom_filename)  # Full path to the DICOM file
        
        if not os.path.exists(full_dose_path):
            print(f"File not found: {full_dose_path}")
            return None
        print(nrrd_filename)
        dose_img = sitk.ReadImage(full_dose_path)
        sitk.WriteImage(dose_img, nrrd_path)
        return nrrd_filename


In [13]:

output_directory_proton = '/home/shahpouriz/Data/DBP_newDATA/nrrd/oneOPTZperPatinets/proton'
output_directory_foton = '/home/shahpouriz/Data/DBP_newDATA/nrrd/oneOPTZperPatinets/foton'

converter = DICOMtoNRRDConverter(directory_path, output_directory_proton, output_directory_foton)
updated_patient_data = converter.convert_and_save()

# # Print the updated patient data
# for patient in updated_patient_data:
#     print(patient)


DBP_OP038_F1_planningCT.nrrd
planningCT
DBP_OP038_F1_planningCT.nrrd
DBP_OP038_F1_repeatedCT1.nrrd
repeatedCT
DBP_OP038_F1_repeatedCT1.nrrd
DBP_OP038_F1_repeatedCT2.nrrd
repeatedCT
DBP_OP038_F1_repeatedCT2.nrrd
DBP_OP038_F1_repeatedCT3.nrrd
repeatedCT
DBP_OP038_F1_repeatedCT3.nrrd
DBP_OP038_F1_repeatedCT4.nrrd
repeatedCT
DBP_OP038_F1_repeatedCT4.nrrd
DBP_OP038_F1_repeatedCT5.nrrd
repeatedCT
DBP_OP038_F1_repeatedCT5.nrrd
DBP_OP038_F1_repeatedCT6.nrrd
repeatedCT
DBP_OP038_F1_repeatedCT6.nrrd
DBP_OP038_F1_repeatedCT7.nrrd
repeatedCT
DBP_OP038_F1_repeatedCT7.nrrd
DBP_OP029_F1_planningCT.nrrd
planningCT
DBP_OP029_F1_planningCT.nrrd
DBP_OP029_F1_repeatedCT1.nrrd
repeatedCT
DBP_OP029_F1_repeatedCT1.nrrd
DBP_OP029_F1_repeatedCT2.nrrd
repeatedCT
DBP_OP029_F1_repeatedCT2.nrrd
DBP_OP029_F1_repeatedCT3.nrrd
repeatedCT
DBP_OP029_F1_repeatedCT3.nrrd
DBP_OP029_F1_repeatedCT4.nrrd
repeatedCT
DBP_OP029_F1_repeatedCT4.nrrd
DBP_OP029_F1_repeatedCT5.nrrd
repeatedCT
DBP_OP029_F1_repeatedCT5.nrrd
DBP_OP029_

KeyboardInterrupt: 

---------
# Extract information regarding nrrd files

In [ ]:

# Save updated_patient_data to a JSON file
output_json_file = output_directory_proton + '/file_info.json' 
with open(output_json_file, 'w') as file:
    json.dump(updated_patient_data, file, indent=4)
    
print(f"Updated patient data saved to {output_json_file}")

Updated patient data saved to /home/shahpouriz/Data/DBP_newDATA/nrrd/oneOPTZperPatinets/proton/file_info.json
